# THIS IS A BOT TO CREATE LABELS FOR EACH PDF ACCORDING TO PILLARS

In [1]:
import pandas as pd
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import openai
import os

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
import numpy as np

## Declare the main path 
1. Create TestSamples folder
2. All pdfs should be saved in TestSamples
3. All excel files provided should be saved in TestSamples

In [2]:
# Change this path according to where you have created the TestSamples folder
main_path = "C:/Users/Alexander/Dropbox/TestSamples"

In [45]:
# Create OPEN-AI KEY
open_ai_key = "paste your key here"

## We create the categories for each group

In [1]:
infrastructure = [
    "Broadband infrastructure", "Internet infrastructure", "Network infrastructure", "Connectivity infrastructure", 
    "Fiber infrastructure", "Broadband connectivity", "Broadband", "digital inclusion", "network connectivity", 
    "Telecommunications", "Mobile operator", "Telecom operator", "Network operator", "Submarine network", 
    "Submarine cable", "Satellite", "Cable landing", "Internet cable", "Submarine fiber", "Submarine connectivity", 
    "virtual landing station", "cross-border connectivity", "international gateway", "Backbone network", 
    "national backbone", "fiber backbone", "microwave backbone", "backhaul network", "Local access network", 
    "Fiber-to-the-home", "FTTH", "WIFI", "public access point", "community center", "local loop network", 
    "universal service fund (USF)", "universal access fund (UAF)", "USAF", "broadband access", "Network coverage", 
    "mobile coverage", "digital gender gap", "gender digital divide", "colocation data center", "cloud storage", 
    "cloud platforms", "data center", "cloud computing", "telecom SOE", "telecom state-owned enterprise", 
    "telecom market liberalization", "open to competition", "telecom license", "mobile license", "IXP", 
    "ICT regulator", "telecom regulator", "interconnection", "Infrastructure sharing", "telecom license", 
    "spectrum management", "telecom tax", "digital tax", "national roaming" 
]
platforms = [
            "Digital Government", "egovernment", "public sector digitization",
            "whole-of-government approach", "integrated government systems", "system integration", "e-government portal", "single window", "one-stop shop", "Change management", "enterprise service bus", "government service bus",
            "National ID", "Civil Registration", "birth and death registration", "birth registration", "death registration", "Identification systems", "authentication system", "verification system", "e-ID", "electronic identification", "biometric ID", "Dynamic Social Registries", "digital certificate", "e-authentication", "single sign-on",
            "interoperability", "enterprise architecture", "interoperability framework", "Open government", "open government data", "open data", "data exchange", "data sharing",
            "Public Financial Management systems", "Tax Management Systems", "Customs Management Systems", "e-Procurement", "Human Resources Management Information System", "financial management information system", "financial management system", "financial information management system", "financial management and information system", "integrated FMIS", "integrated financial management information system", "tax reporting system", "tax payment system",
            "citizen feedback management", "grievance redress mechanism"
            ]

financial_services = [
    "Financial access and usage", "Digital financial product", "Online banking", "Mobile banking", "Payment cards", "Digital credit", "Fintech industry", "Remittance prices", "Indirect costs", "Transaction costs",
    "Fintech",
    "Financial inclusion", "National payments system", "Regtech", "Regulatory technology", "AML/CFT requirements",
    "Nonbank e-money issuance", "Nonbank financial institution", "Online marketplace", "Peer-to-peer lending", "Insurtech",
    "Digital currency", "Central bank digital currency", "Cross-Border payments", "Digital payments", "Government payments", "Full service provider (FSP)", "Payment service provider (PSP)", "Interoperability", "API", "Digital finance solutions", "Cryptocurrency",
    "Cybersecurity", "Data privacy", "Data protection", "Data sharing", "AML/CFT", "KYC",
    "Payment systems", "Payment switch", "Payment gateway", "Fast payments", "Mobile money", "e-money", "Transaction accounts", "Automated clearing house (ACH)", "Electronic funds transfer (EFT)",
    "Credit Reporting System", "Collateral Registry", "Transaction data"
]

businesses = [
    "Digital market", "Digital trade", "Licensing", "Certification standard", "Employment opportunities", "Digital job creation",
    "Incubator", "Accelerator", "Tech park", "Digital innovation ecosystem", "Business ecosystem",
    "Digital startups", "Established digital businesses", "Platform-based businesses", "Data-driven businesses", "Verification mechanism", "Digital technology", "Disruptive technologies",
    "Hardware manufacturing", "Information services", "Software and IT consulting", "Telecommunications",
    "Digital services", "Platform economy", "Sharing economy", "Gig economy",
    "Algorithmic economy", "Precision agriculture", "e-commerce", "Digit commerce", "e-business", "Industry 4.0",
    "Fintech", "E-commerce", "Logistics tech", "Business management tech", "Web services", "Big data and analytics", "Tech hardware", "Edtech", "Social network", "Security tech", "Telecom", "Insurance tech", "Entertainment tech", "Mobility tech", "Health tech"
]

skills = [
    "Digital competencies", "ICT skills", "Digitally competent workforce", "Digitally literate citizens", "Digital Skills Framework", "e-Competence framework",
    "Digital Literacy Global Framework", "Digital Skills Gap", "ICT Curriculum", "Digital Skills Curriculum", "digital skills training", "Digital literacy",
    "Basic digital skills", "Digitally literate", "computer literacy", "IT skills", "digital workforce", "IT workers", "digitally skilled workforce",
    "digital entrepreneurship", "digital start-up", "data analytics skills", "Coding academy", "Coding bootcamps", "ICT professions", "software engineer",
    "AI skills", "big data skills", "STEM skills", "software developer", "application developer", "cybersecurity expert", "DevOps", "Cloud professional",
    "Data Scientists", "Database managers", "digital training for public sector employee", "data analytics for government", "IT government officials",
    "IT officials", "Edtech companies", "Ed-tech companies", "Apprentices programs", "digital training provider", "digital incubator", "digital accelerator",
    "tech hubs", "National Research and Education Network (NREN)"
]

safeguards = [
    "cyber threats", "cyber crime", "computer emergency response team (CERT)", "enterprise security architecture (ESA)", "critical information infrastructure protection (CIIP)",
    "online protection", "CSIRT", "CERT", "cyberattack", "information security", "cyber resilience", "digital security", "online safety", "cybersecurity strategy",
    "Data regulation", "Information privacy", "data protection law", "data privacy law", "access to information", "right to information", "data protection bill",
    "data privacy bill", "information quality and integrity", "data protection authority", "data protection agency", "data commissioner", "PKI", "encryption",
    "data controllers", "data processors", "DP&P", "digital trust", "Child online protection", "Online consumer protection"
]

use_cases = [
    "NREN", "national research and education network", "RREN", "regional research and education network", "campus network", "telemedicine", "medical network", "health network", "rural healthcare", "health care access",
    "education access", "research network", "education network", "education management information system", "health management information system", "hospital management system", "land registration system", "cadastre system",
    "land registration and cadastre system", "social protection management information system", "electronic passport", "e-Passport", "voter registration", "voter ID", "business registry", "corporate registry",
    "company registry", "taxpayer registry", "civil service registry", "public employee registry", "civil servant registry", "beneficiary registry", "social protection system", "social protection registry",
    "vaccination registry", "patient registry", "hospital registry", "healthcare management system", "health care management system", "school registry", "teacher registry", "student registry", "drivers license registry",
    "vehicle registry", "land registry", "land registration", "fixed asset registry", "immovable asset registry", "Payroll Management System", "Asset Management", "Debt Management", "Treasury Single Account", 
    "e-commerce", "electronic commerce", "online commerce", "data-driven agriculture", "Precision farming", "Agtech solutions", "Crop management", "Smart irrigation", "weather stations", "Agricultural sensors", 
    "Farm-to-table traceability", "Remote monitoring", "Livestock management", "precision farming", "e-agriculture", "agritech", "Telemedicine", "Electronic health records", "Wearable devices", "Health analytics",
    "mHealth apps", "Virtual care", "Personalized medicine", "Health data interoperability", "patient management system", "health tech", "education management information system (EMIS)", "Learning management system (LMS)",
    "E-learning platforms", "Open educational resources", "Massive Open Online Courses (MOOCs)", "adaptive learning", "remote learning", "blended learning", "computer labs", "ICT laboratories", "multimedia labs",
    "e-readers", "ICT skills for teachers", "technology for teacher training", "teachers' digital skills", "e-libraries", "ICT in schools", "Intelligent transportation systems", "fleet management",
    "Smart traffic management", "Shared mobility", "e-mobility", "smart mobility", "mobility as a service", "electric vehicle", "car sharing", "electronic fleet management", "ride prediction",
    "Energy management systems", "Smart grid technologies", "smartgrid", "OPGW", "smart factories", "Industrial IoT", "Supply chain traceability", "e-commerce", "smart city", "smart cities",
    "Data analytics", "big data", "open data", "artificial intelligence", "machine learning"
]

# convert each list to a string
infrastructure_str     = ', '.join(infrastructure)
platforms_str          = ', '.join(platforms)
financial_services_str = ', '.join(financial_services)
businesses_str         = ', '.join(businesses)
skills_str             = ', '.join(skills)
safeguards_str         = ', '.join(safeguards)
use_cases_str          = ', '.join(use_cases)

## All pdfs analysis, using all words as one request

In [4]:
openai.api_key = open_ai_key
os.environ["OPENAI_API_KEY"] = open_ai_key

def process_pdf(pdf_path):
    """Processes a PDF file and returns a DataFrame with the results."""

    reader = PdfReader(pdf_path)

    # read data from the file and put them into a variable called raw_text
    raw_text = ''
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        if text:
            raw_text += text

    # We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 
    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 1500,
        chunk_overlap  = 300,
        length_function = len,
    )
    texts = text_splitter.split_text(raw_text)

    # Download embeddings from OpenAI
    embeddings = OpenAIEmbeddings()

    docsearch = FAISS.from_texts(texts, embeddings)

    from langchain.chains.question_answering import load_qa_chain
    from langchain.llms import OpenAI

    chain = load_qa_chain(OpenAI(), chain_type="stuff")

    # define your topic lists
    topic_lists = [infrastructure_str, platforms_str, financial_services_str, 
               businesses_str, skills_str, safeguards_str, use_cases_str]
    topic_names = ['infrastructure', 'platforms', 'financial_services', 
               'businesses', 'skills', 'safeguards', 'use_cases']

    data = {'project': os.path.basename(pdf_path).rstrip(".pdf")}  # Initialize the dictionary with the project name
    
    i = 0
    # process each list
    for topic, name in zip(topic_lists, topic_names):
        # construct the query
        query = f"Do the contents of the document mention any of the following topics: {topic}?"
        # perform the search and process the response
        docs = docsearch.similarity_search(query)
        response = chain.run(input_documents=docs, question=query)
        i = i+1
        print(i)
        print(response)
        #response_dict = json.loads(response)
        #print(str(response['usage']['total_tokens']))
        
        if 'yes' in response.lower():
            data[name] = 1
        else:
            data[name] = 0

    return pd.DataFrame(data, index=[0])


if __name__ == "__main__":
    # CHANGE THIS!!! Get the path to the folder containing the PDF files
    pdf_folder = main_path

    # Process all PDF files in the folder
    pdf_files = os.listdir(pdf_folder)
    df = pd.DataFrame()
    for pdf_file in pdf_files:
        if pdf_file.endswith(".pdf"):
            df = df.append(process_pdf(os.path.join(pdf_folder, pdf_file)), ignore_index=True)

    # Print the DataFrame
    print(df)

1
 No, the document does not mention any of the topics listed.
2
 No, the contents of the document do not mention any of the topics listed.
3
 Yes, the contents of the document mention many of these topics.
4

No, this document does not mention any of those topics.
5
 Yes, this document mentions digital competencies, ICT skills, digitally competent workforce, digitally literate citizens, digital skills training, digital literacy, basic digital skills, digitally literate, computer literacy, IT skills, digital workforce, IT workers, digitally skilled workforce, digital entrepreneurship, digital start-up, data analytics skills, coding academy, coding bootcamps, ICT professions, software engineer, AI skills, big data skills, STEM skills, software developer, application developer, cybersecurity expert, DevOps, cloud professional, data scientists, database managers, digital training for public sector employees, data analytics for government, IT government officials, IT officials, Edtech comp

4
 No, the contents of the document do not mention any of the topics listed.
5
 No, the document does not mention any of these topics.
6
 No, none of these topics are mentioned in the document.
7
 No, the contents of the document do not mention any of the topics listed.
1
 The document does mention broadband infrastructure, Internet infrastructure, Network infrastructure, Connectivity infrastructure, Fixed broadband, Mobile broadband, 5G mobile networks, Broadband enabled use-cases, Network deployment, Internet exchange point, Digital literacy programs, Regulatory framework, Cost reduction measures, Investment attraction measures, Legal and policy framework, Institutional coordination, Regulatory framework, Broadband cost reduction, Investment attraction measures, Network design, Readiness and demand assessments, Deployment, Operations, Maintenance, and Upgrades. It does not mention Fiber infrastructure, Broadband connectivity, Digital inclusion, Network connectivity, Telecommunication

6
 Yes, the document mentions many of the topics listed, including cybersecurity, data regulation, information privacy, data protection law, access to information, right to information, online protection, encryption, data controllers, data processors, and child online protection.
7
 No, none of these topics are mentioned in the document.
1
 No, the contents of the document do not mention any of the topics listed.
2
 No, the document does not mention any of those topics.
3
 No, the contents of the document do not mention any of the topics listed.
4
 No, the document does not mention any of the topics listed.
5
 Yes, the document mentions digital skills training and ICT curriculum.
6
 No, the document does not mention any of those topics.
7
 No, the contents of the document do not mention any of the topics listed.
1
 Yes, the document mentions broadband infrastructure, internet infrastructure, network infrastructure, connectivity infrastructure, fiber infrastructure, broadband connectivi

2
 The document mentions digital government, e-government, public sector digitization, whole-of-government approach, integrated government systems, system integration, e-government portal, single window, one-stop shop, change management, enterprise service bus, government service bus, national ID, civil registration, birth and death registration, identification systems, authentication system, verification system, e-ID, electronic identification, biometric ID, dynamic social registries, digital certificate, e-authentication, single sign-on, interoperability, enterprise architecture, interoperability framework, open government, open government data, open data, data exchange, data sharing, public financial management systems, tax management systems, customs management systems, e-procurement, human resources management information system, financial management information system, financial management system, financial information management system, financial management and information syste

2
 Yes, the document mentions Digital Government, e-government, public sector digitization, whole-of-government approach, integrated government systems, system integration, e-government portal, single window, one-stop shop, Change management, enterprise service bus, government service bus, National ID, Civil Registration, birth and death registration, birth registration, death registration, Identification systems, authentication system, verification system, e-ID, electronic identification, biometric ID, Dynamic Social Registries, digital certificate, e-authentication, single sign-on, interoperability, enterprise architecture, interoperability framework, Open government, open government data, open data, data exchange, data sharing, Public Financial Management systems, Tax Management Systems, Customs Management Systems, e-Procurement, Human Resources Management Information System, financial management information system, financial management system, financial information management syste

...

6
 Yes, the contents of the document mention cyber attack vulnerability, information security, and data protection law.
7
 No, the document does not mention any of the topics listed.
1
 No, these topics are not mentioned in the document.
2
 No, none of these topics are mentioned in the document.
3
 No, the contents of the document do not mention any of the topics listed.
4
 No, the document does not mention any of those topics.
5
 No, the document does not mention any of the topics listed.
6
 No, the contents of the document do not mention any of these topics.
7
 No, none of these topics are mentioned in the contents of the document.
1
 No, the contents of the document do not mention any of the topics listed.
2
 No, none of those topics are mentioned in the document.
3
 No, the contents of the document do not mention any of the topics listed.
4
 No, this document does not mention any of the topics listed.
5
 No, the contents of the document do not mention any of those topics.
6
 No, no

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 06939b6eb9d4439656030ff44619cbe7 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 06939b6eb9d4439656030ff44619cbe7 in your message.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 06939b6eb9d4

1
 No, none of the topics mentioned in the question are mentioned in the document.
2
 Yes, the contents of the document mention some of the topics listed, including e-government, whole-of-government approach, integrated government systems, e-government portal, single window, change management, enterprise service bus, government service bus, National ID, Civil Registration, birth and death registration, Identification systems, authentication system, verification system, e-ID, electronic identification, biometric ID, Dynamic Social Registries, digital certificate, e-authentication, single sign-on, interoperability, enterprise architecture, interoperability framework, Open government, open government data, open data, data exchange, data sharing, Public Financial Management systems, Tax Management Systems, Customs Management Systems, e-Procurement, Human Resources Management Information System, financial management information system, financial management system, financial information mana

7
 No, none of those topics are mentioned in the document.
1
 No, none of those topics are mentioned in the document.
2
 No, none of these topics are mentioned in this document.
3
 Yes, the contents of the document mention many of the topics listed, including Digital Credit, Fintech Industry, Financial Inclusion, Digital Payments, Payment Service Provider (PSP), Interoperability, API, Cybersecurity, Data Privacy, Data Protection, Data Sharing, AML/CFT, KYC, Payment Systems, Payment Switch, Payment Gateway, Mobile Money, E-Money, Transaction Accounts, Automated Clearing House (ACH), Electronic Funds Transfer (EFT), Credit Reporting System, Collateral Registry, and Transaction Data.
4
 No, the document does not mention any of the topics listed.
5
 No, none of these topics are mentioned in the document.
6
 No, none of those topics are mentioned in the document.
7
 No, none of those topics are mentioned in the document.
1
 No, the document does not mention any of these topics.
2
 Yes, the 

6
 No, the contents of the document do not mention any of the topics listed.
7
 No, the document does not mention any of these topics.
1
 No, none of those topics are mentioned in the document.
2
 No, the contents of the document do not mention any of the topics listed.
3
 No, the document does not mention any of the topics listed.
4
 No, the contents of the document do not mention any of the topics listed.
5
 No, the contents of the document do not mention any of the topics listed.
6
 No, the document does not mention any of these topics.
7
 No, none of those topics are mentioned in the document.
         project  infrastructure  platforms  financial_services  businesses  \
0    P160766_PAD               0          0                   1           0   
1    P164416_PAD               0          0                   0           0   
2    P166193_PAD               0          1                   1           0   
3    P166415_PAD               0          0                   0           0   


In [ ]:
df.to_csv(main_path+"/digital_tags.csv", index=False)

# Measuring Accuracy from Predicted Values

## Cleaning created dataset from AI Bot and Julia data

In [8]:
df = pd.read_csv(main_path+"/digital_tags.csv")

# clean project colum
df['project'] = df['project'].str.split('_').str[0]
df.columns

# New column names
new_column_names = ['P-Code', 'infrastructure_pred', 'platforms_pred', 'financial_services_pred',
       'businesses_pred', 'skills_pred', 'safeguards_pred', 'use_cases_pred']

# Assign the new column names to the dataframe
df.columns = new_column_names

# to uppercase 
df['P-Code'] = df['P-Code'].str.upper()

In [9]:
# Clean merging column 
manual_tags = pd.read_excel(main_path+"/Tier1_list.xlsx", engine='openpyxl')
manual_tags['P-Code'] = manual_tags['P-Code'].str.strip()
manual_tags['P-Code'] = manual_tags['P-Code'].str.upper()

# Drop not relevant coliumns
manual_tags = manual_tags.drop(['URL', 'URL.1'], axis=1)
manual_tags

,#,P-Code,Project Approval FY,Region Abbreviation Code,Country Short Name,Project Name,Lending type,Lead GP,Status,Document type,Infrastructure3,PublicSectorPlatform4,FinancialServices5,Businesses6,Skills7,UseCases8
0,1,P160766,2021,AFE,Tanzania,Digital Tanzania Project,IPF,Digital Development,Active,PAD,1,1,0,1,1,0
1,2,P164416,2021,AFE,Burundi,Burundi Skills for Jobs: Women and Youth Project,IPF,Education,Active,PAD,0,0,0,1,1,0
2,3,P166193,2021,AFE,Comoros,Comoros Financial Inclusion Project,IPF,"Finance, Competitiveness and Innovation",Active,PAD,0,0,1,0,0,0
3,4,P166415,2021,AFE,Tanzania,Higher Education for Economic Transformation P...,IPF,Education,Active,PAD,0,1,0,0,1,0
4,5,P167543,2021,AFW,Niger,Niger: Smart Villages for rural growth and dig...,IPF,Digital Development,Active,PAD,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,98,P178667,2024,AFW,Burkina Faso,Support to the Adaptive Social Protection System,IPF,Social Protection & Jobs,Pipeline,PCN,0,1,1,0,0,1
98,99,P179046,2023,LCR,Brazil,Progestão Acre: Public Sector Management Effic...,IPF,Governance,Pipeline,PCN,0,1,0,0,0,1
99,100,P179088,2023,LCR,Brazil,Progestão Tocantins: Public Sector Management ...,IPF,Governance,Pipeline,PCN,0,1,0,0,0,1
100,101,P179108,2023,ECA,Uzbekistan,Uzbekistan Digital Inclusion Project,IPF,Digital Development,Pipeline,PCN,0,0,0,1,1,0


In [10]:
# Clean erros in dataset created from AI bot. 
# P171517 this is on the excel but not as pdf file
# P171513this is as pdf but not in the excel 
# I have double checked and the correct name for the pdf should have been "P171517"
df['P-Code'].replace('P171513', 'P171517', inplace=True)

In [11]:
tags_orig_pred = manual_tags.merge(df, on='P-Code')
tags_orig_pred

,#,P-Code,Project Approval FY,Region Abbreviation Code,Country Short Name,Project Name,Lending type,Lead GP,Status,Document type,...,Businesses6,Skills7,UseCases8,infrastructure_pred,platforms_pred,financial_services_pred,businesses_pred,skills_pred,safeguards_pred,use_cases_pred
0,1,P160766,2021,AFE,Tanzania,Digital Tanzania Project,IPF,Digital Development,Active,PAD,...,1,1,0,0,0,1,0,1,1,0
1,2,P164416,2021,AFE,Burundi,Burundi Skills for Jobs: Women and Youth Project,IPF,Education,Active,PAD,...,1,1,0,0,0,0,0,1,0,0
2,3,P166193,2021,AFE,Comoros,Comoros Financial Inclusion Project,IPF,"Finance, Competitiveness and Innovation",Active,PAD,...,0,0,0,0,1,1,0,1,0,0
3,4,P166415,2021,AFE,Tanzania,Higher Education for Economic Transformation P...,IPF,Education,Active,PAD,...,0,1,0,0,0,0,0,0,0,0
4,5,P167543,2021,AFW,Niger,Niger: Smart Villages for rural growth and dig...,IPF,Digital Development,Active,PAD,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,98,P178667,2024,AFW,Burkina Faso,Support to the Adaptive Social Protection System,IPF,Social Protection & Jobs,Pipeline,PCN,...,0,0,1,0,0,0,0,0,0,0
98,99,P179046,2023,LCR,Brazil,Progestão Acre: Public Sector Management Effic...,IPF,Governance,Pipeline,PCN,...,0,0,1,0,0,0,0,0,0,0
99,100,P179088,2023,LCR,Brazil,Progestão Tocantins: Public Sector Management ...,IPF,Governance,Pipeline,PCN,...,0,0,1,0,0,0,0,0,0,0
100,101,P179108,2023,ECA,Uzbekistan,Uzbekistan Digital Inclusion Project,IPF,Digital Development,Pipeline,PCN,...,1,1,0,1,0,0,1,1,0,0


In [12]:
# Fix Some non numerical values in columns 
tags_orig_pred["FinancialServices5"].replace('\xa0', 0, inplace=True)
tags_orig_pred["Businesses6"].replace('\xa0', 0, inplace=True)
tags_orig_pred["Skills7"].replace('\xa0', 0, inplace=True)

In [13]:
# Create true and predicted vectors
y_true1 = tags_orig_pred.Infrastructure3
y_pred1 = tags_orig_pred.infrastructure_pred

y_true2 = tags_orig_pred.PublicSectorPlatform4
y_pred2 = tags_orig_pred.platforms_pred

y_true3 = tags_orig_pred.FinancialServices5
y_pred3 = tags_orig_pred.financial_services_pred

y_true4 = tags_orig_pred.Businesses6
y_pred4 = tags_orig_pred.businesses_pred

y_true5 = tags_orig_pred.Skills7
y_pred5 = tags_orig_pred.skills_pred

y_true6 = tags_orig_pred.UseCases8
y_pred6 = tags_orig_pred.use_cases_pred

### Accuracy Score

In [14]:
print('Infrastructure:{:.2f}'.format(accuracy_score(y_true1, y_pred1)))
print('Public Sector Plataform:{:.2f}'.format(accuracy_score(y_true2, y_pred2 )))
print('Financial Services:{:.2f}'.format(accuracy_score(y_true3, y_pred3 )))
print('Businessess:{:.2f}'.format(accuracy_score(y_true4, y_pred4 )))
print('Skills:{:.2f}'.format(accuracy_score(y_true5, y_pred5 )))
print('UseCases:{:.2f}'.format(accuracy_score(y_true6, y_pred6 )))

Infrastructure:0.85
Public Sector Plataform:0.48
Financial Services:0.79
Businessess:0.77
Skills:0.72
UseCases:0.60


### Classification Report

In [15]:
import pandas as pd
from sklearn.metrics import classification_report

# Define your labels
labels = ['Infrastructure', 'Public Sector Platform', 'Financial Services', 'Businesses', 'Skills', 'Use Cases']

# Define your y_true and y_pred pairs
pairs = [(y_true1, y_pred1), (y_true2, y_pred2), (y_true3, y_pred3), (y_true4, y_pred4), (y_true5, y_pred5), (y_true6, y_pred6)]

# Calculate classification reports for each set of predictions
markdown_tables = []
for label, (y_true, y_pred) in zip(labels, pairs):
    report = classification_report(y_true, y_pred, output_dict=True)
    df_report = pd.DataFrame(report).transpose()
    markdown_table = df_report[['precision', 'recall', 'f1-score']].to_markdown()
    markdown_tables.append(f"## {label}\n{markdown_table}\n")

# Print all markdown tables
for table in markdown_tables:
    print(table)

## Infrastructure
|              |   precision |   recall |   f1-score |
|:-------------|------------:|---------:|-----------:|
| 0            |    0.84507  | 0.9375   |   0.888889 |
| 1            |    0.870968 | 0.710526 |   0.782609 |
| accuracy     |    0.852941 | 0.852941 |   0.852941 |
| macro avg    |    0.858019 | 0.824013 |   0.835749 |
| weighted avg |    0.854718 | 0.852941 |   0.849294 |

## Public Sector Platform
|              |   precision |   recall |   f1-score |
|:-------------|------------:|---------:|-----------:|
| 0            |    0.30137  | 0.916667 |   0.453608 |
| 1            |    0.931034 | 0.346154 |   0.504673 |
| accuracy     |    0.480392 | 0.480392 |   0.480392 |
| macro avg    |    0.616202 | 0.63141  |   0.479141 |
| weighted avg |    0.782878 | 0.480392 |   0.492658 |

## Financial Services
|              |   precision |   recall |   f1-score |
|:-------------|------------:|---------:|-----------:|
| 0            |    0.863014 | 0.851351 |   0.857143

C:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alexander\anaconda3\envs\tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

# analysis in subgroups

In [16]:
tags_orig_pred_act = tags_orig_pred[tags_orig_pred['Status'] == "Active"]
tags_orig_pred_act

,#,P-Code,Project Approval FY,Region Abbreviation Code,Country Short Name,Project Name,Lending type,Lead GP,Status,Document type,...,Businesses6,Skills7,UseCases8,infrastructure_pred,platforms_pred,financial_services_pred,businesses_pred,skills_pred,safeguards_pred,use_cases_pred
0,1,P160766,2021,AFE,Tanzania,Digital Tanzania Project,IPF,Digital Development,Active,PAD,...,1,1,0,0,0,1,0,1,1,0
1,2,P164416,2021,AFE,Burundi,Burundi Skills for Jobs: Women and Youth Project,IPF,Education,Active,PAD,...,1,1,0,0,0,0,0,1,0,0
2,3,P166193,2021,AFE,Comoros,Comoros Financial Inclusion Project,IPF,"Finance, Competitiveness and Innovation",Active,PAD,...,0,0,0,0,1,1,0,1,0,0
3,4,P166415,2021,AFE,Tanzania,Higher Education for Economic Transformation P...,IPF,Education,Active,PAD,...,0,1,0,0,0,0,0,0,0,0
4,5,P167543,2021,AFW,Niger,Niger: Smart Villages for rural growth and dig...,IPF,Digital Development,Active,PAD,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,P178563,2022,LCR,Brazil,RECOVERING LEARNING LOSSES FROM COVID-19 PANDE...,PforR,Education,Active,PAD,...,0,1,1,0,0,0,0,1,0,0
78,79,P178609,2023,LCR,Argentina,Strengthening data infrastructure to close the...,IPF,Digital Development,Active,PAD,...,0,1,0,1,1,0,0,1,1,0
79,80,P178658,2023,AFE,Mozambique,Access to Finance & Economic Opportunities Pro...,IPF,"Finance, Competitiveness and Innovation",Active,PAD,...,0,0,0,0,0,1,0,0,0,0
80,81,P178801,2022,MNA,Lebanon,Lebanon Digital Procurement,IPF,Governance,Active,ProjectPaper,...,0,0,1,0,1,0,0,0,0,0


In [17]:
# Create true and predicted vectors
y_true1 = tags_orig_pred_act.Infrastructure3
y_pred1 = tags_orig_pred_act.infrastructure_pred

y_true2 = tags_orig_pred_act.PublicSectorPlatform4
y_pred2 = tags_orig_pred_act.platforms_pred

y_true3 = tags_orig_pred_act.FinancialServices5
y_pred3 = tags_orig_pred_act.financial_services_pred

y_true4 = tags_orig_pred_act.Businesses6
y_pred4 = tags_orig_pred_act.businesses_pred

y_true5 = tags_orig_pred_act.Skills7
y_pred5 = tags_orig_pred_act.skills_pred

y_true6 = tags_orig_pred_act.UseCases8
y_pred6 = tags_orig_pred_act.use_cases_pred

In [18]:
### Accuracy Score

print('Infrastructure:{:.2f}'.format(accuracy_score(y_true1, y_pred1)))
print('Public Sector Plataform:{:.2f}'.format(accuracy_score(y_true2, y_pred2 )))
print('Financial Services:{:.2f}'.format(accuracy_score(y_true3, y_pred3 )))
print('Businessess:{:.2f}'.format(accuracy_score(y_true4, y_pred4 )))
print('Skills:{:.2f}'.format(accuracy_score(y_true5, y_pred5 )))
print('UseCases:{:.2f}'.format(accuracy_score(y_true6, y_pred6 )))

### Classification Report

Infrastructure:0.86
Public Sector Plataform:0.46
Financial Services:0.82
Businessess:0.74
Skills:0.67
UseCases:0.62


In [19]:
tags_orig_pred.to_excel(main_path+'/tags_orig_pred.xlsx', index=False)

# False Negative

In [21]:
tags_orig_pred.columns

Index(['#', 'P-Code', 'Project Approval FY', 'Region Abbreviation Code',
       'Country Short Name', 'Project Name', 'Lending type', 'Lead GP',
       'Status', 'Document type', 'Infrastructure3', 'PublicSectorPlatform4',
       'FinancialServices5', 'Businesses6', 'Skills7', 'UseCases8',
       'infrastructure_pred', 'platforms_pred', 'financial_services_pred',
       'businesses_pred', 'skills_pred', 'safeguards_pred', 'use_cases_pred'],
      dtype='object')

In [23]:
false_negatives = ((tags_orig_pred['Infrastructure3'] == 1) & (tags_orig_pred['infrastructure_pred'] == 0)).sum()
false_negatives

11

In [25]:
false_negatives = tags_orig_pred[(tags_orig_pred['Infrastructure3'] == 1) & (tags_orig_pred['infrastructure_pred'] == 0)]
false_negative_ids = false_negatives['P-Code']
false_negative_ids

0     P160766
8     P169698
10    P169945
27    P172647
30    P173373
36    P174620
42    P175218
65    P177005
77    P178563
81    P178824
86    P176733
Name: P-Code, dtype: object

In [43]:
pred_cols = ['infrastructure_pred', 'platforms_pred', 'financial_services_pred',
             'businesses_pred', 'skills_pred', 'safeguards_pred', 'use_cases_pred']

orig_cols = ['Infrastructure3', 'PublicSectorPlatform4', 'FinancialServices5', 
             'Businesses6', 'Skills7', 'UseCases8']

# Initialize an empty DataFrame to store false negative IDs
false_negatives_df = pd.DataFrame()

# Loop over each pair of columns
for pred_col, orig_col in zip(pred_cols, orig_cols):
    # Filter the DataFrame for false negatives in this pair of columns
    false_negatives = tags_orig_pred[(tags_orig_pred[orig_col] == 1) & (tags_orig_pred[pred_col] == 0)]
    
    # Get the IDs of the false negatives and add them to the DataFrame
    false_negatives_df[pred_col + '_false_negatives'] = false_negatives['P-Code']

# Reset the DataFrame's index
false_negatives_df = false_negatives_df.reset_index(drop=True)

In [44]:
false_negatives_df

,infrastructure_pred_false_negatives,platforms_pred_false_negatives,financial_services_pred_false_negatives,businesses_pred_false_negatives,skills_pred_false_negatives,safeguards_pred_false_negatives
0,P160766,P160766,NaN,P160766,NaN,NaN
1,P169698,NaN,NaN,NaN,NaN,P169698
2,P169945,NaN,NaN,NaN,NaN,NaN
3,P172647,P172647,NaN,NaN,NaN,P172647
4,P173373,NaN,NaN,P173373,NaN,NaN
5,P174620,NaN,NaN,NaN,P174620,NaN
6,P175218,P175218,NaN,NaN,NaN,P175218
7,P177005,NaN,NaN,P177005,P177005,NaN
8,P178563,P178563,NaN,NaN,NaN,P178563
9,P178824,P178824,NaN,NaN,NaN,P178824


In [45]:
false_negatives_df.to_excel(main_path+'/false_negatives_p_code.xlsx', index=False)

In [46]:
df_no_na = false_negatives_df.apply(lambda series: series.dropna().reset_index(drop=True))
df_no_na

,infrastructure_pred_false_negatives,platforms_pred_false_negatives,financial_services_pred_false_negatives,businesses_pred_false_negatives,skills_pred_false_negatives,safeguards_pred_false_negatives
0,P160766,P160766,NaN,P160766,P174620,P169698
1,P169698,P172647,NaN,P173373,P177005,P172647
2,P169945,P175218,NaN,P177005,NaN,P175218
3,P172647,P178563,NaN,NaN,NaN,P178563
4,P173373,P178824,NaN,NaN,NaN,P178824
5,P174620,P176733,NaN,NaN,NaN,NaN
6,P175218,NaN,NaN,NaN,NaN,NaN
7,P177005,NaN,NaN,NaN,NaN,NaN
8,P178563,NaN,NaN,NaN,NaN,NaN
9,P178824,NaN,NaN,NaN,NaN,NaN


In [47]:
# False Positive
pred_cols = ['infrastructure_pred', 'platforms_pred', 'financial_services_pred',
             'businesses_pred', 'skills_pred', 'safeguards_pred', 'use_cases_pred']

orig_cols = ['Infrastructure3', 'PublicSectorPlatform4', 'FinancialServices5', 
             'Businesses6', 'Skills7', 'UseCases8']

# Initialize an empty DataFrame to store false negative IDs
false_positives_df = pd.DataFrame()

# Loop over each pair of columns
for pred_col, orig_col in zip(pred_cols, orig_cols):
    # Filter the DataFrame for false negatives in this pair of columns
    false_positives = tags_orig_pred[(tags_orig_pred[orig_col] == 0) & (tags_orig_pred[pred_col] == 1)]
    
    # Get the IDs of the false negatives and add them to the DataFrame
    false_positives_df[pred_col + '_false_positives'] = false_positives['P-Code']

# Reset the DataFrame's index
false_positives_df = false_positives_df.reset_index(drop=True)
false_positives_df

,infrastructure_pred_false_positives,platforms_pred_false_positives,financial_services_pred_false_positives,businesses_pred_false_positives,skills_pred_false_positives,safeguards_pred_false_positives
0,P172350,NaN,NaN,NaN,NaN,P172350
1,P174684,NaN,NaN,NaN,P174684,NaN
2,P176108,NaN,P176108,NaN,NaN,NaN
3,P179108,NaN,NaN,NaN,NaN,NaN


In [48]:
false_positives_df.to_excel(main_path+'/false_positives_p_code.xlsx', index=False)